In [1]:
import gradio as gr
import pickle
import requests
import os
from constants import ROOT_DIR, USERS_DIR

# Initialize environment variables
AZ_CODE = os.environ.get("AZ_CODE")
API_URL = os.environ.get("API_URL")


# Functions to load user IDs and get recommendations
def load_users(path: str, n_users: int = 25) -> list:
    """
    Load user IDs from the pickle file and sample a specified number of unique user IDs to return as a sorted list.
    """
    with open(path, "rb") as file:
        df = pickle.load(file)
    user_ids = df["user_id"].sample(n_users).unique().tolist()
    # Add non existing users IDS
    user_ids.extend([*range(1000000, 1000010, 2)])
    return sorted(user_ids)


def get_recommendations(
    user_id: int, api_url: str = API_URL, az_code: str = AZ_CODE
) -> tuple:
    """
    Get article recommendations for a given user ID from the Azure Function API and return the top 5 recommended articles and the recommendation method.
    If the API call fails, return an error message.
    """
    params = {"user_id": user_id, "code": az_code}
    try:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            recommendations = data.get("recommendations", [])
            method = data.get("method", "Méthode inconnue")
            return recommendations, method
        else:
            return f"Erreur API: {response.status_code} - {response.text}", ""
    except Exception as e:
        return f"Erreur lors de la requête: {e}", ""


user_ids = load_users(USERS_DIR)

iface = gr.Interface(
    fn=get_recommendations,
    inputs=gr.Dropdown(
        choices=user_ids, label="Sélectionnez un identifiant utilisateur"
    ),
    outputs=[
        gr.Textbox(label="Articles recommandés (top 5)"),
        gr.Textbox(
            label="Méthode de recommandation (à masquer pour les utilisateurs finaux)"
        ),
    ],
    title="My Content: Recommandation d'articles",
    description="Sélectionnez un identifiant utilisateur pour obtenir 5 recommandations d'articles.",
    submit_btn="Recommander",
    clear_btn="Effacer",
    flagging_mode="never",
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import os
import gradio as gr
import requests


AZ_CODE = os.environ.get("AZ_CODE")
API_URL = os.environ.get("API_URL")
user_ids = load_users(os.path.join(ROOT_DIR, "data", "merged_df.pickle"))


def get_recommendations(user_id):
    params = {"user_id": user_id, "code": AZ_CODE}
    try:
        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            print(data)
            return data.get("recommendations", [])
        else:
            return f"Erreur API: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Erreur lors de la requête: {e}"


iface = gr.Interface(
    fn=get_recommendations,
    inputs=gr.Dropdown(choices=user_ids, label="Sélectionnez un user_id"),
    outputs=gr.Textbox(label="Articles recommandés (top 5)"),
    title="Recommandation d'articles via Azure Functions",
    description="Sélectionnez un utilisateur pour obtenir 5 recommandations d'articles.",
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
